In [11]:
%load_ext autoreload
%autoreload 2

In [50]:
#export
import torch, numpy as np, time, k1lib, math, matplotlib.pyplot as plt
from k1lib import getFirst
from functools import partial

In [70]:
#export
class Callback:
    """Represents a callback. Define specific functions
    inside to intercept certain parts of the training
    loop. Can reference Learner's attrs directly
    using `self` like this:
    >>> self.opt # actually accessing Learner.opt, if `opt` is not defined
    
    You can also set Learner's attributes like this:
    >>> self.learner.xb = self.xb[None]
    This takes x batch of learner, unsqueeze it at
    the 0 position, then sets the x batch again.
    
    Normally, you will define a subclass of this and
    define specific intercept functions, but if you
    want to create a throwaway callback, then do this:
    >>> Callback().withCheckpoint("startRun", lambda: print("start running"))"""
    def __init__(self, name=None): self.learner = None; self._cbs = None; self._name = name or self.__class__.__name__
    def _injectCbs(self, cbs): self._cbs = cbs; return self
    def _injectLearner(self, learner): self.learner = learner; return self
    def __getattr__(self, attr):
        if hasattr(self.learner, attr): return getattr(self.learner, attr)
        else: raise AttributeError
    def _repr_html_(self): return f"Callback `<code>{self._name}</code>`"
    def withCheckpoint(self, checkpoint:str, f:callable): setattr(self, checkpoint, lambda: f(self)); return self
    def detach(self):
        """Detaches from the parent Callbacks object.
        Remember to save the Callback somewhere to
        reference it if needed."""
        self._cbs.remove(self._name); return self

In [67]:
#export
class Callbacks:
    """Can reference each individual callbacks directly
    by the class name."""
    def __init__(self): self.cbs = []; self.learner = None; self.dict = {}
    def injectLearner(self, learner):
        """Intended to be used by Learner only. Injects
        into all callbacks and update learner's dict."""
        self.learner = learner; learner._updateCbsDict()
        [cb._injectLearner(learner) for cb in self.cbs]; return self
    def append(self, cb, name=None):
        """Adds a callback to the collection."""
        name = name or cb._name
        if name in self.dict:
            i = 0
            while f"{name}{i}" in self.dict: i += 1
            name = f"{name}{i}"
        cb._name = name; self.dict[name] = cb; self.__dict__[name] = cb
        self.cbs.append(cb._injectLearner(self.learner)._injectCbs(self))
        if self.learner is not None: self.learner._updateCbsDict()
        return self
    def remove(self, name):
        """Removes a callback from the collection."""
        if name not in self.dict: return print(f"Callback `{name}` not found")
        self.cbs.remove(self.dict[name]); del self.dict[name]; return self
    def __call__(self, checkpoint, *args, **kwargs):
        """Intended to be used by Learner only to fire
        off events."""
        output = False
        for cb in self.cbs:
            if hasattr(cb, checkpoint):
                if getattr(cb, checkpoint)(*args, **kwargs) != None:
                    output = True
        return output
    def __getattr__(self, attr):
        if attr in self.dict: return self.dict[attr]
        else: raise AttributeError
    def _repr_html_(self): return f"<pre>Callbacks:<ul>{''.join([f'<li>{cbName}</li>' for cbName in self.dict])}</ul></pre>"
    @staticmethod
    def standard(advanced=True):
        """Standard callbacks. Include advanced callbacks
        if `advanced` is True. They can impact performance,
        so it's possible to switch them off."""
        cbs = Callbacks().withProgressBar().withLoss()
        if advanced: cbs.withHookModule().withHookParam()
        return cbs

In [5]:
#export
def _withInspectLoss(self, f:callable):
    """Expected `f` to take in 1 float."""
    class InspectLoss(Callback):
        def endLoss(self): f(self.loss.detach())
    return self.append(InspectLoss())
Callbacks.withInspectLoss = _withInspectLoss
def _withCuda(self):
    self.withModifyBatch(lambda xb, yb: (xb.cuda(), yb.cuda()))
    class Cuda(Callback):
        def startRun(self): self.model.cuda()
    return self.append(Cuda())
Callbacks.withCuda = _withCuda

In [6]:
#export
def _withInspectBatch(self, f:callable):
    """Expected `f` to take in 2 tensors."""
    class InspectBatch(Callback):
        def startBatch(self): f(self.xb, self.yb)
    return self.append(InspectBatch())
Callbacks.withInspectBatch = _withInspectBatch
def _withModifyBatch(self, f:callable):
    """Modifies xb and yb on the fly. Expected `f`
    to take in 2 tensors and return 2 tensors."""
    class ModifyBatch(Callback):
        def startBatch(self): self.learner.xb, self.learner.yb = f(self.xb, self.yb)
    return self.append(ModifyBatch())
Callbacks.withModifyBatch = _withModifyBatch

In [7]:
#export
def _withInspectOutput(self, f:callable):
    """Expected `f` to take in 1 tensor."""
    class InspectOutput(Callback):
        def endPass(self): f(self.y)
    return self.append(InspectOutput())
Callbacks.withInspectOutput = _withInspectOutput
def _withModifyOutput(self, f:callable):
    class ModifyOutput(Callback):
        def endPass(self): self.learner.y = f(self.y)
    return self.append(ModifyOutput())
Callbacks.withModifyOutput = _withModifyOutput

In [8]:
#export
def _withProgressBar(self):
    """Displays the current progress, epoch and batch
    while running."""
    class ProgressBar(Callback):
        def startRun(self): self.startTime = time.time()
        def startBatch(self):
            self.elapsedTime = time.time() - self.startTime
            self.learner.progress = (self.batch / self.nBatches + self.epoch) / self.epochs
            print(f"\rProgress: {round(100 * self.progress)}%, epoch: {self.epoch}/{self.epochs}, batch: {self.batch}/{self.nBatches}, elapsed: {np.round(self.elapsedTime, 2)}s         ", end="")
    return self.append(ProgressBar())
Callbacks.withProgressBar = _withProgressBar

In [64]:
#export
_docsLoss = """
Records losses after each batch.

Methods:
- Loss.plot()
- Loss.epoch.plot()

Fields:
- Loss.train
- Loss.valid
- Loss.epoch.train
- Loss.epoch.valid"""
def _withLoss(self):
    def nonEmptyList(_list): return [0] if _list == [] else _list
    class Loss(Callback):
        def __init__(self):
            super().__init__()
            def _plotStuff(obj):
                plt.figure(figsize=(10, 3), dpi=100)
                plt.subplot(1, 2, 1); plt.plot(obj.train); plt.title("Train loss")
                plt.subplot(1, 2, 2); plt.plot(obj.valid); plt.title("Valid loss"); plt.show()
            self.train = []; self.valid = [] # all stats all times
            self.epoch = k1lib.Object.fromDict({"train": [], "valid": []}) # average stats for each epoch
            self.plot = partial(_plotStuff, self); self.epoch.plot = partial(_plotStuff, self.epoch)
            self._trainLosses = []; self._validLosses = []
        def endLoss(self):
            if self.model.training:
                self._trainLosses.append(self.loss/self.xb.shape[0])
            else: self._validLosses.append(self.loss/self.xb.shape[0])
        def endEpoch(self):
            self.train.extend(self._trainLosses); self.epoch.train.append(np.mean(nonEmptyList(self._trainLosses)))
            self.valid.extend(self._validLosses); self.epoch.valid.append(np.mean(nonEmptyList(self._validLosses)))
            self._trainLosses = []; self._validLosses = []
    Loss.__doc__ = _docsLoss; return self.append(Loss())
_withLoss.__doc__ = _docsLoss; Callbacks.withLoss = _withLoss

In [52]:
#export
def _withParamFinder(self, param="lr"):
    """Automatically finds out the right value for
    a specific parameter"""
    class ParamFinder(Callback):
        def __init__(self, param):
            self.exps = np.linspace(-6, 2, 50); self.values = 10**self.exps; self.idx = 0
            self.bestLoss = 1e9; self.param = param; self.losses = []
        def startBatch(self):
            if self.idx >= len(self.exps): raise k1lib.CancelRunException
            for paramGroup in self.opt.param_groups: paramGroup[self.param] = self.values[self.idx]
            self.idx += 1
        def endLoss(self):
            loss = self.loss/self.xb.shape[0]
            if loss < self.bestLoss: self.bestLoss = loss
            if loss > self.bestLoss * 10: raise k1lib.CancelRunException
            self.losses.append(loss)
        def run(self): self.learner.run(10); self.plot(); return self
        def plot(self):
            if len(self.losses) == 0: print("Has not run param finder yet.")
            else:
                plt.plot(self.values[:len(self.losses)], self.losses)
                plt.xscale("log"); plt.xlabel(self.param); plt.ylabel("Loss"); plt.show()
    return self.append(ParamFinder(param))
Callbacks.withParamFinder = _withParamFinder

In [53]:
#export
_docsHookModule="""
Records means and std of output of individual
modules on both forward and backward pass. Erases
info before each run.

Can pass through additional forward and backward
callbacks. There are several methods for that:
    .withForwardHook(). This literally just appends the
        hook to the variable `.forwardFns`
    .withBackwardHook(). This literally just appends the
        hook to the variable `.backwardFns`
    .withHook(). Just calls the 2 functions above

You can manipulate `.forwardFns` and `.backwardFns`
directly. But if you need quality of life stuff, here
are more methods:
    .clearHooks()

After a learner is created and bound with a Callbacks,
you can add a hook like this:
>>> learner.HookModule.withMeanRecorder()

There are a few built in hooks that you can check out:
    .withMeanRecorder()
    .withStdRecorder()
    .withMinRecorder()
    .withMaxRecorder()
    .withHistRecorder()

By default, this will analyze `learner.model`, but
you can change that like this:
>>> learner.HookModule.module = <intended nn.Module object>

After a run, you can access a module's data by exploring these:
>>> learner.HookModule[i].forward.<field>
>>> learner.HookModule[i].backward.<field>
`<field>` is any field you pass to `data` in the hook
you passed to `.withHook`

If your field is just a simple list of numbers, you
can plot all values in all modules using `.plot()`:
>>> learner.HookModule.plot()"""
def _withHookModule(self):
    def hook(fns, *args): [fn(*args) for fn in fns]
    class HookModule(Callback):
        def __init__(self):
            super().__init__()
            self._handles = [] # the hook handles returned by pytorch at startRun, so that 
            self._moduleDatas = []; self._moduleNames = []
            self.forwardFns = []; self.backwardFns = [] # lsit of forward and backward hooks
            self.cleanFns = []; self._module = None
        def withForwardHook(self, hook:callable, name=None):
            """Adds a hook to the forward pass. See `.withHook()` for more information"""
            hook.name = name; self.forwardFns += [hook]; return self
        def withBackwardHook(self, hook:callable, name=None):
            """Adds a hook to the backward pass. See `.withHook()` for more information"""
            hook.name = name; self.backwardFns += [hook]; return self
        def withHook(self, hook:callable, name=None):
            """Adds a hook to both the forward and backward pass.
            Params:
                `hook` function is expected to take in these parameters: (data, module, inp, out)
                    data: the injected dependency for you to store stuff.
                        Initially, `data` is an empty object, so you have to
                        check whether it has your field like so:
                        >>> if not hasattr(data, "min"): data.min = float("inf")
                        >>> data.min = torch.min(data.min, out)

                        If you do not do this, then undefined variables will
                        automatically be an empty list, so that this is fine:
                        >>> data.max.append() # okay

                        Later on, you can do things like:
                        >>> HookModule[i].forward.min
                        and get the data you saved from the hook.
                    module: the module this function hooks into. Please
                        refer to `torch.nn.Module.register_forward_hook()` to
                        know more.
                    inp: input (or grad of input) to the module
                    out: output (or grad of output) to the module
                `name`: custom name for the function for nice displaying
            """
            return self.withForwardHook(hook, name).withBackwardHook(hook, name)
        def clearHooks(self): self.forwardFns = []; self.backwardFns = []; self.cleanFns = []; return self
        def withModule(self, module): self._module = module; return self
        @property
        def module(self): return self._module or self.model
        @module.setter
        def module(self, module): self._module = module
        def withMeanRecorder(self): return self.withHook(lambda data, m, inp, out: data.means.append(getFirst(out).data.mean().item()), "mean")
        def withStdRecorder(self): return self.withHook(lambda data, m, inp, out: data.stds.append(getFirst(out).data.std().item()), "std")
        def withMinRecorder(self): return self.withHook(lambda data, m, inp, out: data.mins.append(getFirst(out).data.min().item()), "min")
        def withMaxRecorder(self): return self.withHook(lambda data, m, inp, out: data.maxs.append(getFirst(out).data.max().item()), "max")
        def withHistRecorder(self, bounds=[-50, 50], bins:int=100, fieldName:str="hist"):
            """Records a histogram of the output and places in
            data.`fieldName`. If you want multiple recordings,
            you have to use different `fieldName`s.
            
            `bounds` and `bins` are just fields passed to `torch.histc()`.
            
            A good use case for multiple field names may be:
            >>> .withHistRecorder(bounds=[-50, 50], "histWide")
            >>> .withHistRecorder(bounds=[-5, 5], "histNarrow")"""
            def cleanPart(data):
                hists = getattr(data, fieldName)
                if len(hists) == 0: return
                setattr(data, f"{fieldName}Img", torch.stack(hists).T)
            self.cleanFns.append(lambda forwardData, backwardData: (cleanPart(forwardData), cleanPart(backwardData)))
            hook = lambda data, m, inp, out: getattr(data, fieldName).append(k1lib.getFirst(out).data.histc(bins, *bounds))
            return self.withHook(hook, fieldName)
        def plotHist(self, fieldName:str="hist"):
            """Plots the recorded histogram with a `fieldName`
            specified in `.withHistRecorder()`."""
            def _plot(objF):
                n = len(self); plt.figure(dpi=100, figsize=(10, math.ceil(n/4) * 2))
                for i, (data, name) in enumerate(zip(self, self._moduleNames)):
                    plt.subplot(math.ceil(n/4), 4, i+1); plt.title(f"{i}.{name}"); plt.axis("off")
                    try: plt.imshow(getattr(objF(data), f"{fieldName}Img"))
                    except: pass
                plt.show()
            print("Forward"); _plot(lambda data: data.forward); print("Backward"); _plot(lambda data: data.backward)
        def plotHistRatio(self, _slice:slice=slice(49, 51), fieldName="hist"):
            """Plots how much elements are in a certain bin
            `_slice` compared to every slice. This may be
            useful in determining if the gradients stay at
            0 too much."""
            def _plot(objF):
                n = len(self); plt.figure(dpi=100, figsize=(10, math.ceil(n/4) * 2))
                for i, (data, name) in enumerate(zip(self, self._moduleNames)):
                    plt.subplot(math.ceil(n/4), 4, i+1); plt.title(f"{i}.{name}")
                    try: a = getattr(objF(data), f"{fieldName}Img"); plt.ylim(0, 1); plt.xticks([], []); plt.yticks([], []); plt.plot(a[_slice].sum(dim=0) / a.sum(dim=0))
                    except: pass
                plt.show()
            print("Forward"); _plot(lambda data: data.forward); print("Backward"); _plot(lambda data: data.backward)
        def startRun(self):
            self._moduleDatas = []; self._moduleNames = []; self._handles = []
            for idx, module in enumerate(self.module):
                data = k1lib.Object.fromDict({"forward": k1lib.Object().withAutoDeclare(lambda: []),
                                             "backward": k1lib.Object().withAutoDeclare(lambda: []),
                                             "name": module.__class__.__name__})
                self._moduleDatas.append(data); self._moduleNames.append(module.__class__.__name__)
                self._handles.append(module.register_forward_hook(partial(hook, self.forwardFns, data.forward)))
                self._handles.append(module.register_backward_hook(partial(hook, self.backwardFns, data.backward)))
        def endRun(self):
            for moduleData in self._moduleDatas:
                for cleanFn in self.cleanFns:
                    cleanFn(moduleData.forward, moduleData.backward)
            for handle in self._handles:
                handle.remove()
        def __getitem__(self, idx): return self._moduleDatas[idx]
        def __len__(self): return len(self._moduleDatas)
        def _repr_html_(self):
            f = f"<ul>{''.join([f'<li>{fn.name or str(fn)}</li>' for fn in self.forwardFns])}</ul>"
            b = f"<ul>{''.join([f'<li>{fn.name or str(fn)}</li>' for fn in self.backwardFns])}</ul>"
            #s = f", {len(self[0].forward.means)} means and stds each" if len(self) > 0 else ""
            n = f"<ol start='0'>{''.join([f'<li><code>{name}</code></li>' for name in self._moduleNames])}</ol>"
            return f"{super()._repr_html_()}: {len(self)} modules:{n}Forward hooks:{f}Backward hooks:{b}"
        def plot(self, fields=["means", "stds"], _slice:slice=slice(None, None, None), logScale:bool=False):
            logScale = "log" if logScale else "linear"
            def _plot(i, objF, title):
                plt.subplot(len(fields), 2, i)
                for module in self: obj = objF(module); plt.plot(range(len(obj))[_slice], obj[_slice])
                plt.title(title); plt.yscale(logScale)
            plt.figure(figsize=(10, 3*len(fields)), dpi=100)
            for i, field in enumerate(fields):
                _plot(i*2+1, lambda m: getattr(m.forward, field), f"Forward {field}")
                _plot(i*2+2, lambda m: getattr(m.backward, field), f"Backward {field}")
            plt.figlegend(self._moduleNames, loc='center right'); plt.show()
    HookModule.__doc__ = _docsHookModule; return self.append(HookModule().withMeanRecorder().withStdRecorder())
_withHookModule.__doc__ = _docsHookModule; Callbacks.withHookModule = _withHookModule

In [68]:
#export
_docsHookParam="""
Records means and stds of all parameters.

Methods:
- HookParam.plot()

Fields:
- HookParam[i].means
- HookParam[i].stds"""
def _withHookParam(self):
    class HookParam(Callback, k1lib.ListContainer):
        def __init__(self):
            super().__init__()
            self._setField("_parameterDatas"); self._parameters = []; self._paramNames = []
        def startRun(self):
            self._parameters = list(self.model.parameters())
            self._parameterDatas = [k1lib.Object.fromDict({"means": [], "stds": []}) for param in self._parameters]
            self._paramNames = [param[0] for param in self.model.named_parameters()]
        def startBatch(self):
            for data, param in zip(self._parameterDatas, self._parameters):
                data.means.append(param.detach().mean().item())
                data.stds.append(param.detach().std().item())
        def _repr_html_(self):
            s = f", {len(self[0].means)} means and stds each" if len(self) > 0 else ""
            names = "".join([f"<li><code>{name}</code></li>" for name in self._paramNames])
            return f"{super()._repr_html_()}: {len(self)} params{s}:<ol start='0'>{names}</ol>"
        def plot(self, _slice:slice=slice(None, None, None), logScale=False):
            logScale = "log" if logScale else "linear"
            def _plot(i, objF, title):
                plt.subplot(1, 2, i)
                for m in self: obj = objF(m); plt.plot(range(len(obj))[_slice], obj[_slice])
                plt.title(title); plt.yscale(logScale)
            plt.figure(figsize=(10, 3), dpi=100); _plot(1, lambda m: m.means, "Means"); _plot(2, lambda m: m.stds, "Stds")
            plt.figlegend(self._paramNames, loc='right'); plt.show()
    HookParam.__doc__ = _docsHookParam; return self.append(HookParam())
_withHookParam.__doc__ = _docsHookParam; Callbacks.withHookParam = _withHookParam

In [71]:
!exportnb callbacks.ipynb

/home/kelvin/repos/labs/k1lib/k1lib
Current dir: 0, /home/kelvin/repos/labs/k1lib/export.py
File: /home/kelvin/repos/labs/k1lib/k1lib/callbacks.py
running bdist_wheel
running build
installing to build/bdist.linux-x86_64/wheel
running install
running install_egg_info
running egg_info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing top-level names to k1lib.egg-info/top_level.txt
reading manifest file 'k1lib.egg-info/SOURCES.txt'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
Copying k1lib.egg-info to build/bdist.linux-x86_64/wheel/k1lib-0.1.0-py3.8.egg-info
running install_scripts
creating build/bdist.linux-x86_64/wheel/k1lib-0.1.0.dist-info/WHEEL
creating 'dist/k1lib-0.1.0-py3-none-any.whl' and adding 'build/bdist.linux-x86_64/wheel' to it
adding 'k1lib-0.1.0.dist-info/METADATA'
adding 'k1lib-0.1.0.dist-info/WHEEL'
adding 'k1lib-0.1.0.dist-info/top_level.txt'
adding 'k1lib-0.1.0.dist-info/RECORD'
removing build/bdist.linux-x86_

In [47]:
cbs = Callbacks.standard()

In [49]:
cbs.ProgressBar._repr_html_()

AttributeError: 

In [30]:
cbs.withProgressBar()

In [57]:
class A:
    def __init__(self): pass
    def fa(self, x): return x + 3

In [58]:
def fb(self, x): return x + 5

In [59]:
A.fb = fb

In [60]:
a = A()

In [61]:
a.fb(3)

8